In [3]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
import math

In [ ]:
OdeParams = namedtuple('OdeParams', ['x_inf', 'tau_x'])


class LinearDynamics(metaclass=ABCMeta):

    @abstractmethod
    def params(self, state) -> OdeParams:
        pass
    

RateParams = namedtuple('RateParams', ['A', 'xo', 'kappa'])
    

class Rate(metaclass=ABCMeta):
    
    def __init__(self, params):
        self.params = params
        
    def __getattr__(self, attr):
        try:
            value = getattr(self.params, attr)
        except AttributeError():
            raise AttributeError()
        return value
    
    @abstractmethod
    def __call__(self, x):
        pass
    
    
class ExpLinRate(Rate):
    
    def __init__(self, params, tol_x):
        super().__init__(self, params)
        self.tol_x = tol_x
    
    def __call__(self, x):
        Δx = x - self.xo
        if abs(Δx) < self.tol_x:
            z = self.A * self.kappa
        else:
            x1 = self.kappa * Δx
            if x1 > 0:
                e_x = math.exp(-x1)
                z = Δx * e_x / (1 - e_x)
            else:
                z = Δx / (math.exp(x1) - 1)
        return z * self.A
    

class SigmoidRate(Rate):
    
    def __call__(self, x):
        Δx = (x - self.xo) / self.kappa
        if abs(Δx) < 0:
            z = 1 / (1 + math.exp(Δx))
        else:
            e_x = math.exp(-Δx)
            z = e_x / (1 + e_x)
        return z * self.A


class ExpRate(Rate):
    
    def __call__(self, x):
        Δx = (x - self.xo) / self.kappa
        return math.exp(Δx) * self.A
    
    
class MarkovDynamics(LinearDynamics):
    
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def params(self, state):
        α = self.a(state)
        β = self.b(state)
        τ = 1 / (α + β)
        return OdeParams(α * τ, τ)
    
    
class GatingVariable:
    
    def __init__(self, dyn, value=0):
        self.dyn = dyn
        self.value = value
        
    def update(self, v, integrator):
        params = self.dyn.params(v)
        self.value = integrator.solve(params, self.value)

        
class Conductance:
    
    def __init__(self, g_max, gates):
        self.g_max = g_max
        self.gates = gates

    @property
    def value(self):
        pass
    
    
class ConductanceBuilder
        
        
    
class Integrator(metaclass=ABCMeta):
    
    def __init__(self, delta):
        self.delta = delta
    
    @abstractmethod
    def solve(self, dyn, xo):
        pass
    

class ForwardEuler(Integrator):
    
    def solve(self, dyn, xo):
        return xo + self.delta * (dyn.x_inf - xo) / tau_x
        

class AnalyticLinear(Integrator):
    
    def __init__(self, delta):
        self.delta = delta
        
    def solve(self, dyn, xo):
        return dyn.x_inf + (xo - dyn.x_inf) * math.exp(-self.delta / dyn.tau_x)